In [25]:
import pandas as pd 

df = pd.read_csv("food_recommed.csv")
df['해산물'] =  pd.to_numeric(df['해산물'])

user_input = {
        "remaining_calories": 520,
        "remaining_carbs": 50,
        "remaining_protein": 20,
        "remaining_fat": 15,
        "meat_count": 2,
        "veg_count": 1,
        "seafood_count": 0}

# [하루 권장 칼로리 - 현재 섭취 칼로리, 섭취가능 탄수화물량, 섭취가능 단백질량, 섭취가능 지방량, 고기 섭취 횟수,채소 섭취 횟수,해산물 섭취 횟수]

In [26]:
max_count = max_count = max(user_input["meat_count"], user_input["veg_count"], user_input["seafood_count"]) 
# 고기, 채소, 해산물 섭취 횟수 중 가장 많이 섭취한 횟수

total_count = user_input["meat_count"] + user_input["veg_count"] + user_input["seafood_count"]

# 고기를 제일 많이 먹은 사람에게 또 고기 메뉴를 추천하는 것을 방지하기 위해 가중치 부여
meat_weight = (max_count - user_input["meat_count"]) / total_count if total_count != 0 else 1
veg_weight = (max_count - user_input["veg_count"]) / total_count if total_count != 0 else 1
seafood_weight = (max_count - user_input["seafood_count"]) / total_count if total_count != 0 else 1

In [27]:
# 칼로리 및 탄단지를 기준으로 음식을 추천하기 때문에 가중치 계산
cal_score = 1 - abs(df["에너지(kcal)"] - user_input["remaining_calories"]) / user_input["remaining_calories"]
carb_score = 1 - abs(df["탄수화물(g)"] - user_input["remaining_carbs"]) / user_input["remaining_carbs"]
protein_score = 1 - abs(df["단백질(g)"] - user_input["remaining_protein"])/ user_input["remaining_protein"]
fat_score = 1 - abs(df["지방(g)"] - user_input["remaining_fat"])/ user_input["remaining_fat"]

# 고기, 채소, 해산물 섭취 횟수를 기반으로 가중치 합계 계산
category_weight = (
    df["고기"] * meat_weight
    + df["채소"] * veg_weight
    + df["해산물"] * seafood_weight
)

# 고기,채소,해산물 섭취 횟수와 칼로리 및 탄단지 가중치 계산 점수를 기반으로 각 음식이 얼마나 적합한지 계산
# 칼로리 및 탄단지 가중치 계산 점수에 더 비중을 두기 위해 조금 더 높은 숫자를 곱함
df["Recommendation_Score"] = 0.62 * (cal_score + carb_score + protein_score + fat_score) / 4 + 0.38 * category_weight

In [29]:
def food_recommendation(user_input):

    # 남는 칼로리와 탄단지량
    remaining_calories = user_input["remaining_calories"]
    remaining_carbs = user_input["remaining_carbs"]
    remaining_protein = user_input["remaining_protein"]
    remaining_fat = user_input["remaining_fat"]

    # 추천 점수와 칼로리를 기반으로 데이터셋 추천 음식 목록 데이터셋 정렬
    sorted_foods = df.sort_values(by=["Recommendation_Score", "에너지(kcal)"], ascending=[False, True])

    selected_foods = []
    total_calories, total_carbs, total_protein, total_fat = 0, 0, 0, 0

    for _, row in sorted_foods.iterrows():
        food_calories = row["에너지(kcal)"]
        food_carbs = row["탄수화물(g)"]
        food_protein = row["단백질(g)"]
        food_fat = row["지방(g)"]

        if (
            total_calories + food_calories <= remaining_calories
            and total_carbs + food_carbs <= remaining_carbs
            and total_protein + food_protein <= remaining_protein
            and total_fat + food_fat <= remaining_fat
        ): 
            # 만약 더 섭취할 수 있는 영양소량보다 기존 섭취 영양소량 + 현재 음식 섭취 시 영양소량이 작거나 같을 때만
            # (= 하루 권장 섭취량을 넘어가지 않는 선에서 음식 추천)
            selected_foods.append(row["음 식 명"])
            total_calories += food_calories
            total_carbs += food_carbs
            total_protein += food_protein
            total_fat += food_fat

    # 가장 추천점수가 높은 음식명 반환
    return [selected_foods[0]] if selected_foods else ["No food found"]

In [30]:
def get_recommendation0(user_input: dict):
    return food_recommendation(user_input)

In [31]:
get_recommendation0(user_input)

['연어 샐러드          ']